**Table of contents**<a id='toc0_'></a>    
- 1. [Imports](#toc1_)    
- 2. [Load Data           ](#toc2_)    
  - 2.1. [Hyperparameters](#toc2_1_)    
- 3. [Life cycle income](#toc3_)    
- 4. [Convergence](#toc4_)    
- 5. [Life-cycle profiles](#toc5_)    
- 6. [Transfer](#toc6_)    
- 7. [Speed of Convergence           ](#toc7_)    
  - 7.1. [With DP](#toc7_1_)    
  - 7.2. [New LR](#toc7_2_)    
  - 7.3. [Without DP](#toc7_3_)    
- 8. [(Consumption) Euler Errors](#toc8_)    
- 9. [Table](#toc9_)    
  - 9.1. [DP](#toc9_1_)    
  - 9.2. [Full](#toc9_2_)    
- 10. [Policy functions](#toc10_)    
  - 10.1. [1D](#toc10_1_)    
    - 10.1.1. [Chose where to plot policy](#toc10_1_1_)    
    - 10.1.2. [Generate policies in levels](#toc10_1_2_)    
    - 10.1.3. [Plot](#toc10_1_3_)    
  - 10.2. [8D](#toc10_2_)    
    - 10.2.1. [Choose where to plot policy](#toc10_2_1_)    
    - 10.2.2. [Get policy in levels](#toc10_2_2_)    
    - 10.2.3. [Plot policy](#toc10_2_3_)    
- 11. [More on convergence](#toc11_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
%load_ext autoreload
%autoreload 2

## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [11]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE' # without this python may crash when plotting from matplotlib
import sys

from IPython.display import display, HTML

import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid":True,"grid.color": "black","grid.alpha":"0.25","grid.linestyle": "--"})
plt.rcParams.update({'font.size':14})
plt.rcParams.update({'font.family':'serif'})
import matplotlib.lines as mlines

from copy import deepcopy

In [12]:
parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)
from plot_funcs import load_all, compute_transfer, convergence_plot, transfer_plot, train_specs
from DurablesModel import DurablesModelClass
from model_funcs import compute_d_and_c_from_action

In [13]:
import EconDLSolvers

## 2. <a id='toc2_'></a>[Load Data](#toc2_)            [&#8593;](#toc0_)

In [14]:
folder_load = '../output/'
folder = '../output/results'
algonames = ['DeepSimulate','DeepFOC','DeepVPD']
Ds = [1,2,3,8]

In [15]:
algolabels = {
    'DeepSimulate':'DeepSimulate',
    'DeepFOC':'DeepFOC',
    'DeepVPD':'DeepVPD+',
    'DeepVPD-FOC':'DeepVPD++',
}

In [ ]:
models = load_all(folder_load,'DurablesModel')

**Change baseline:**

In [17]:
# for D in [1,2,3]:
#     Dstr = f'{D}D'
#     models[('DeepVPD',Dstr,'fixedLR')] = models[('DeepVPD',Dstr)]
#     models[('DeepVPD',Dstr)] = models[('DeepVPD',Dstr,'altLR')]

### 2.1. <a id='toc2_1_'></a>[Hyperparameters](#toc0_)

In [ ]:
do_display = False
for D in Ds:
    if do_display: print(f'D = {D}')
    models_ = {k:v for k,v in models.items() if int(k[1][0]) == D and len(k) == 2}
    train_specs(models_,do_display=do_display,folder=folder,filename=f'DurablesModel_train_specs_{D}D')

## 3. <a id='toc3_'></a>[Life cycle income](#toc0_)

In [19]:
model = models[('DP','1D')]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,3))

ax.plot(model.par.kappa)

ax.set_xlabel('period $t$')
ax.set_ylabel('$\\kappa_t$')

filepath = f"{folder}/DurablesModel_kappa.svg"
plt.savefig(filepath, bbox_inches='tight')

## 4. <a id='toc4_'></a>[Convergence](#toc0_)

In [ ]:
for D in Ds:
    #for algoname,extra in [(algoname,None) for algoname in algonames] + [('DeepVPD','fixedLR')]:
    for algoname,extra in [(algoname,None) for algoname in algonames]:
        
        if D == 8 and not extra is None: continue 
        
        print(f'D = {D}, algoname = {algoname}',end='')
        if extra is not None: 
            print(f' {extra}',end='')
        else:
            print('')

        model = models[(algoname,f'{D}D')] if extra is None else models[(algoname,f'{D}D',extra)]
        postfix = f'_DurablesModel_{algoname}_{D}D'
        if extra is not None: postfix += f'_{extra}'

        EconDLSolvers.convergence_plot(model,
                         y_fac=10_000,ylabel='transfer, bp. of initial cash-on-hand',
                         folder=folder,postfix=postfix,close_fig=False)
        
        plt.show()

In [ ]:
for D in [1,2,8]:
        
    model = models[('DeepFOC',f'{D}D')]
    k_max = model.info['iter']

    x = [model.info[('k_time',k)]/60 for k in range(k_max) if ('policy_loss',k) in model.info and model.info[('k_time',k)]/60 > 1]    
    y = [model.info[('policy_loss',k)] for k in range(k_max) if ('policy_loss',k) in model.info and model.info[('k_time',k)]/60 > 1]

    x_ = []
    y_ = []
    best_yi = np.inf
    for xi,yi in zip(x,y):
        if yi < best_yi:
            x_.append(xi)
            y_.append(yi)
            best_yi = yi

    fig, ax = plt.subplots(1,1,figsize=(6,4))
    # ax.set_title(f'DeepFOC - {D}D')
    ax.plot(x_,y_,'-o',ms='2',color=colors[0])
    ax.set_xlabel('time (m)')
    ax.set_yscale('log')

    fig.tight_layout()
    fig.savefig(f'{folder}/DurablesModel_DeepFOC_{D}D_policy_loss.svg')

    plt.show()


## 5. <a id='toc5_'></a>[Life-cycle profiles](#toc0_)

In [ ]:
for D in [1,2,3]:
    for algoname in ['DP'] + algonames:

        if algoname == 'DP' and D == 3: continue

        print(algoname,D)

        fig = plt.figure(figsize=(12,4))

        ax = fig.add_subplot(1,2,1)
        if not algoname == 'DP':
            label = algolabels[algoname]
            ax.plot(np.mean(models[(f'{algoname}',f'{D}D')].sim.states[:,:,0],axis=1),label=label,color=colors[0], lw=2)
            if D <= 2:
                ax.plot(np.mean(models[('DP',f'{D}D','fine')].sim.states[:,:,0],axis=1),label='NEGM+',color=colors[0], lw=2,ls='--')
            else:
                ax.plot(np.mean(models[('DP',f'{D}D','rough')].sim.states[:,:,0],axis=1),label='NEGM-',color=colors[0], lw=2,ls='--')
        else:
            ax.plot(np.mean(models[('DP',f'{D}D')].sim.states[:,:,0],axis=1),label='NEGM',color=colors[0], lw=2,ls='-')
            ax.plot(np.mean(models[('DP',f'{D}D','rough')].sim.states[:,:,0],axis=1),label='NEGM-',color=colors[0], lw=2,ls='--')
            ax.plot(np.mean(models[('DP',f'{D}D','fine')].sim.states[:,:,0],axis=1),label='NEGM+',color=colors[0], lw=2,ls='-.')
        
        ax.axvline(x=model.par.T_retired, lw=2,color='black', linestyle='--')
        ax.set_xlabel('period, $t$')
        ax.set_title('cash-on-hand')
        ax.legend()

        ax = fig.add_subplot(1,2,2)
        if not algoname == 'DP':
            for k in range(D):
                ax.plot(np.mean(models[(f'{algoname}',f'{D}D')].sim.states[:,:,2+k],axis=1),label=f'durable {1+k}',color=colors[k], lw=2)
                if D <= 2:
                    ax.plot(np.mean(models[('DP',f'{D}D')].sim.states[:,:,2+k],axis=1),color=colors[k], lw=2,ls='--')
                else:               
                    ax.plot(np.mean(models[('DP',f'{D}D','rough')].sim.states[:,:,2+k],axis=1),color=colors[k], lw=2,ls='--')
        else:
            for k in range(D):
                ax.plot(np.mean(models[('DP',f'{D}D')].sim.states[:,:,2+k],axis=1),label=f'durable {1+k}',color=colors[k], lw=2,ls='-')
                ax.plot(np.mean(models[('DP',f'{D}D','rough')].sim.states[:,:,2+k],axis=1),color=colors[k], lw=2,ls='--')
                ax.plot(np.mean(models[('DP',f'{D}D','fine')].sim.states[:,:,2+k],axis=1),color=colors[k], lw=2,ls='-.')
                
        ax.axvline(x=model.par.T_retired,lw=2, label='retirement',color='black', linestyle='--')
        ax.set_xlabel('period, $t$')
        
        if D == 1:
            ax.set_title('durable')
        else:
            ax.set_title('durables')
            ax.legend()

        fig.tight_layout()
        fig.savefig(f'{folder}/DurablesModel_lcp_{D}D_{algoname}.svg')

        plt.show()

In [ ]:
basenames = ['DeepSimulate']

for basename in basenames:
    for D in [1,2,3,8]:
        for algoname in algonames: 

            if algoname == basename: continue

            label = algolabels[algoname]
            print(f'{algoname}, {D}D [basename: {basename}]')

            fig = plt.figure(figsize=(12,10))

            ax = fig.add_subplot(2,1,1)
            ax.plot(np.mean(models[(f'{algoname}',f'{D}D')].sim.states[:,:,0],axis=1),label=label,color=colors[0],linewidth=2)
            ax.plot(np.mean(models[(f'{basename}',f'{D}D')].sim.states[:,:,0],axis=1),label=basename,color=colors[0],linewidth=2,ls='--')
            ax.axvline(x=model.par.T_retired, linewidth=2,color='black', linestyle='--')
            
            ax.set_ylim([0.9,2.5])
            ax.set_xlabel('period, $t$')
            ax.set_title('cash-on-hand')
            ax.legend()

            ax = fig.add_subplot(2,1,2)
            for k in range(D):
                delta = models[(f'{algoname}',f'{D}D')].par.delta[k]
                ax.plot(np.mean(models[(f'{algoname}',f'{D}D')].sim.states[:,:,2+k],axis=1),label=f'{1+k}',color=colors[k], linewidth=2)
                ax.plot(np.mean(models[(f'{basename}',f'{D}D')].sim.states[:,:,2+k],axis=1),color=colors[k], linewidth=2,ls='--')
            
            ax.axvline(x=model.par.T_retired,linewidth=2,color='black', linestyle='--')
            if D < 3:
                ax.set_ylim([0,1.0])
            elif D == 3:
                ax.set_ylim([0,0.7])
            elif D == 8:
                ax.set_ylim([0,0.3])
            ax.set_xlabel('period, $t$')
            ax.set_title('durables')
            ax.legend(ncols=5,fontsize=12)

            fig.tight_layout()

            filepath = f"{folder}/DurablesModel_lcp_{D}D_{algoname}_base{basename}.svg"
            fig.savefig(filepath)

            if D <= 3:
                plt.close(fig)
                display(HTML(f'<a href="{filepath}">{filepath}</a>'))
            else:
                plt.show()

In [ ]:
basenames = ['DeepSimulate']

for basename in basenames:
    for D in [1,2,3,8]:
        for algoname in algonames: 

            if algoname == basename: continue

            label = algolabels[algoname]
            print(f'{algoname}, {D}D [basename: {basename}]')

            fig = plt.figure(figsize=(12,10))

            ax = fig.add_subplot(2,1,1)
            m_base = models[(f'{basename}',f'{D}D')].sim.states[:,:,0]
            m_algo = models[(f'{algoname}',f'{D}D')].sim.states[:,:,0]
            y = [np.corrcoef(m_base[t],m_algo[t])[0,1] for t in range(model.par.T)]

            ax.plot(y,label=label,color=colors[0],linewidth=2)
            ax.axvline(x=model.par.T_retired, linewidth=2,color='black', linestyle='--')
            
            ax.set_ylim([0.8,1.01])
            ax.set_xlabel('period, $t$')
            ax.set_title('cash-on-hand')
            ax.legend()

            ax = fig.add_subplot(2,1,2)
            for k in range(D):
                d_base = models[(f'{basename}',f'{D}D')].sim.states[:,:,2+k]
                d_algo = models[(f'{algoname}',f'{D}D')].sim.states[:,:,2+k]   
                y = [np.corrcoef(d_base[t],d_algo[t])[0,1] for t in range(model.par.T)]             
                ax.plot(y,label=f'{1+k}',color=colors[k], linewidth=2)
            
            ax.axvline(x=model.par.T_retired,linewidth=2,color='black', linestyle='--')
            ax.set_ylim([0.8,1.01])
            ax.set_xlabel('period, $t$')
            ax.set_title('durables')
            ax.legend(ncols=5,fontsize=12)

            fig.tight_layout()

            filepath = f"{folder}/DurablesModel_lcp_corr_{D}D_{algoname}_base{basename}.svg"
            fig.savefig(filepath)

            if D <= 2:
                plt.close(fig)
                display(HTML(f'<a href="{filepath}">{filepath}</a>'))
            else:
                plt.show()

## 6. <a id='toc6_'></a>[Transfer](#toc0_)

In [ ]:
Ds = [1]
for D in Ds:
    transfer_plot('DurablesModel',models,algonames,D,folder=folder)

## 7. <a id='toc7_'></a>[Speed of Convergence](#toc4_2_)            [&#8593;](#toc0_)

### 7.1. <a id='toc7_1_'></a>[With DP](#toc0_)

In [27]:
xlim = [0.1,1000]
ylim = [-1000,100]

In [ ]:
for D in [1,2,3]:
    
    for DPstr in ['rough','','fine']:

        Dstr = f'{D}D'
        postfix = f'_{Dstr}'
        print(Dstr,DPstr)

        if DPstr == '': 
            if  ('DP',Dstr) not in models: continue
            DP = models[('DP',Dstr)]
        else:
            if  ('DP',Dstr,DPstr) not in models: continue
            DP = models[('DP',Dstr,DPstr)]

        if DPstr == 'fine':
            DP_name = '+'
        elif DPstr == 'rough':
            DP_name = '-'
        else:
            DP_name = ''

        if not DPstr == '': postfix += f'_{DPstr}'

        specs = {(algoname,Dstr):algolabels[algoname] for algoname in algonames}

        convergence_plot('DurablesModel',models,specs,DP=DP,do_transfer=True,DP_name=f'NEGM{DP_name}',
                            xlim=xlim,ylim=ylim,folder=folder,postfix=postfix)  

### 7.2. <a id='toc7_2_'></a>[New LR](#toc0_)

In [29]:
# for D in [1,2,3]:
    
#     for DPstr in ['rough','','fine']:

#         Dstr = f'{D}D'
#         postfix = f'_{Dstr}'
#         print(Dstr,DPstr)

#         if DPstr == '': 
#             if  ('DP',Dstr) not in models: continue
#             DP = models[('DP',Dstr)]
#         else:
#             if  ('DP',Dstr,DPstr) not in models: continue
#             DP = models[('DP',Dstr,DPstr)]

#         if DPstr == 'fine':
#             DP_name = '+'
#         elif DPstr == 'rough':
#             DP_name = '-'
#         else:
#             DP_name = ''

#         if not DPstr == '': postfix += f'_{DPstr}'

#         postfix += f'_lr'
#         specs = {}
#         specs[('DeepVPD',Dstr)] = f'DeepVPD (adjusted lr)'
#         specs[('DeepVPD',Dstr,'fixedLR')] = f'DeepVPD (fixed lr)'

#         convergence_plot('DurablesModel',models,specs,DP=DP,do_transfer=True,DP_name=f'NEGM{DP_name}',
#                             xlim=xlim,ylim=ylim,folder=folder,postfix=postfix)  

### 7.3. <a id='toc7_3_'></a>[Without DP](#toc0_)

In [ ]:
Ds = [8]
ylim = [-50,-44]
for D in Ds:
    
    Dstr = f'{D}D'
    print(Dstr)

    specs = {(algoname,Dstr):algolabels[algoname] for algoname in algonames}

    convergence_plot('DurablesModel',models,specs,DP=None,do_transfer=False,DP_name='NEGM',
                     xlim=xlim,ylim=ylim,
                     folder=folder,postfix=f'_{Dstr}')
    
    convergence_plot('DurablesModel',models,specs,DP=None,do_transfer=False,
                     xlim=xlim,ylim=[-45.50,-45.40],
                     folder=folder,postfix=f'_{Dstr}_zoom')         

### DeepVPD-FOC

In [31]:
# D = 8
# Dstr = f'{D}D'
# specs = {}
# specs[('DeepExplore',Dstr)] = 'DeepExplore'
# specs[('DeepVPD',Dstr)] = algolabels['DeepVPD']
# specs[('DeepVPD',Dstr,'FOC')] = algolabels['DeepVPD-FOC']
# ylim = [-50,-44]

# convergence_plot('DurablesModel',models,specs,DP=None,do_transfer=False,DP_name='NEGM',
#                     xlim=xlim,ylim=ylim,
#                     folder=folder,postfix=f'_{Dstr}_DeepVPD_FOC')

## 8. <a id='toc8_'></a>[(Consumption) Euler Errors](#toc0_)


In [ ]:
def compute_mean_euler_DP(key,model):

    par = model.par
    savings_indicator = (model.sim.states_pd[:model.par.T_retired,:,0] > 1e-4)
    euler_error = model.sim.euler_error[:par.T_retired]
    euler_error = euler_error[savings_indicator]
    J = np.isclose(np.abs(euler_error),0.0)
    model.info['euler_errors'] = np.log10(np.abs(euler_error[~J]))
    mean_euler_error = model.info['mean_euler_error'] = np.mean(model.info['euler_errors'])
    print(f'{str(key):30s} = {mean_euler_error:.2f} [{np.mean(J):.4f} share of euler errors are zero]')
    
for key,model in models.items():
    compute_mean_euler_DP(key,model)

In [ ]:
Ds = [1,2,3,8]
do_display = True
algonames = ['DeepSimulate','DeepFOC','DeepVPD']

for D in Ds:
    print(f'{D = }')
    for i,algoname in enumerate(algonames):

        if D <= 3:
            DP = models[('DP',f'{D}D','fine')] if D < 3 else models[('DP',f'{D}D','rough')]
            _ = DP.info['euler_errors']
        else:
            DP = None

        label = algolabels[algoname]

        fig,ax = plt.subplots(1,1,figsize=(9,6))

        if not DP is None:
            ax.hist(DP.info['euler_errors'],bins=100,alpha=0.5,color='grey',density=True,label='EGM')
            ax.axvline(x=DP.info['mean_euler_error'],color='grey',linestyle='--',label=f'EGM, mean')   

        model = models[(f'{algoname}',f'{D}D')]
        ax.hist(model.info['euler_errors'],bins=100,alpha=0.5,color=colors[i],density=True,label=label)
        ax.axvline(x=model.info['mean_euler_error'],color=colors[i],linestyle='--',label=f'{label}, mean')
     
        ax.set_xlabel('Euler error')
        ax.set_ylabel('density')
        ax.set_xlim([-6,0])
        ax.set_ylim([0,1.4])
        ax.legend(loc='upper left')
        
        filepath = f"{folder}/DurablesModel_euler_error_{D}D_{algoname}.svg"
        fig.savefig(filepath,bbox_inches='tight')

        if do_display:
            plt.show()
        else:
            plt.close(fig)
            display(HTML(f'<a href="{filepath}">{filepath}</a>'))

In [ ]:
Ds = [8]
do_display = True
algonames = ['DeepSimulate','DeepFOC','DeepVPD']

fig,ax = plt.subplots(1,1,figsize=(9,6))
for i,algoname in enumerate(algonames):

    model = models[(f'{algoname}',f'{D}D')]
    label = algolabels[algoname]
    ax.hist(model.info['euler_errors'],bins=100,alpha=0.5,color=colors[i],density=True,label=label)
    ax.axvline(x=model.info['mean_euler_error'],color=colors[i],linestyle='--',label=f'{label}, mean')
    
ax.set_xlabel('Euler error')
ax.set_ylabel('density')
ax.set_xlim([-6,0])
ax.set_ylim([0,1.4])
ax.legend(loc='upper left')

filepath = f"{folder}/DurablesModel_euler_error_{D}D.svg"
fig.savefig(filepath,bbox_inches='tight')

if do_display:
    plt.show()
else:
    plt.close(fig)
    display(HTML(f'<a href="{filepath}">{filepath}</a>'))

## 9. <a id='toc9_'></a>[Table](#toc0_)

### 9.1. <a id='toc9_1_'></a>[DP](#toc0_)

In [35]:
def create_table_DP(models,labels):

    # a. define column and row titles
    columns = labels    
    rows = [
        '$R$', 
        'Euler-error', 
        '',
        'Time (m)', 
        '',
        'Nn', 
        'Nm', 
        'Np', 
        'Nm-pd', 
        'Nm (keep)', 
        '',
        'GB', 
    ]

    # b. create with NaN values
    df = pd.DataFrame(index=rows,columns=columns).fillna('-')
    for col in columns:
        df.loc['',col] = ''

    # c. fill in values
    for model,label in zip(models,labels):
        df.loc['$R$',label] = f"{model.sim.R:.4f}"
        if 'mean_euler_error' in model.info:
            df.loc['Euler-error',label] = f"{model.info['mean_euler_error']:.2f}"
        df.loc['Time (m)',label] = f"{model.info['time']/60:.2f}"
        for key in ['Nn','Nm','Np','Nm_pd','Nm_keep']:
            if key == 'Nm_keep':
                df.loc['Nm (keep)',label] = model.egm.Nm_keep
            elif key == 'Nm_pd':
                df.loc['Nm-pd',label] = model.egm.Nm_pd
            else:
                df.loc[key,label] = model.egm.__dict__[key]
        df.loc['GB',label] = f"{model.info['memory_usage']:.2f}"

    return df

In [ ]:
# a. select
models_DP = []
labels = []
for D in [1,2,3]:

    models_DP.append(models[('DP',f'{D}D','rough')])
    labels.append(f'{D}D-')    

    try:
        models_DP.append(models[('DP',f'{D}D')])
        labels.append(f'{D}D')
    except Exception:
        pass

    try:
        models_DP.append(models[('DP',f'{D}D','fine')])
        labels.append(f'{D}D+')
    except Exception:
        pass

# b. table
df = create_table_DP(models_DP,labels)
display(df)

# c.save
filename = f'{folder}/DurablesModel_table_DP.tex'
df.style.to_latex(filename,hrules=True)  

### 9.2. <a id='toc9_2_'></a>[Full](#toc0_)

In [68]:
def create_table(models,labels,DP=None,DP_name='NEGM'):

    # define column and row titles
    columns = [DP_name] + labels
    
    rows = [
        'Life-time reward, $R$', 
        'Transfer (bp.)', 
        'Euler-error', 
        '',
        'Total time (m)', 
        '\,\,\,update NN', 
        '\,\,\,simulation: training sample', 
        '\,\,\,simulation: out-of-sample $R$', 
        'Iterations', 
        'Avg. policy epochs', 
        'Avg. value epochs',
    ]

    # create with NaN values
    df = pd.DataFrame(index=rows,columns=columns).fillna('-')  # using '-' to represent missing df for display purposes
    for col in columns: df.loc['',col] = ''

    do_DP = DP is not None

    if do_DP: df.loc['Life-time reward, $R$', DP_name] = f'{DP.sim.R:.4f}'
    for model,label in zip(models,labels):
       df.loc['Life-time reward, $R$', label] = f"{model.sim.R:.4f}"

    if do_DP:

        df.loc['Transfer (bp.)', DP_name] = 0
        for model,label in zip(models,labels):

            model = model
            
            R = model.sim.R
            R_transfer = deepcopy(DP.sim.R_transfer)
            transfer = compute_transfer(R_transfer,DP.egm.transfer_grid,R,do_extrap=False)
            transfer_mean = transfer
            
            if not np.isnan(transfer_mean):
                df.loc['Transfer (bp.)', label] = f"{100**2*transfer_mean:.0f}"
            else:
                df.loc['Transfer (bp.)', label] = '-'

    if do_DP: df.loc['Euler-error', DP_name] = f"{DP.info['mean_euler_error']:.2f}"
    for model,label in zip(models,labels):
        model = model
        df.loc['Euler-error', label] = f"{model.info['mean_euler_error']:.2f}"
    
    if do_DP: df.loc['Total time (m)', DP_name] = f"{DP.info['time']/60:.2f}"
    for model,label in zip(models,labels):
        df.loc['Total time (m)', label] = f"{model.info['time']/60:.2f}"

    df.loc['\,\,\,update NN',DP_name] = '-'
    for model,label in zip(models,labels):
        time_rel = model.info['time.update_NN'] / model.info['time']
        df.loc['\,\,\,update NN', label] = f"{100*time_rel:.1f}\%"

    df.loc['\,\,\,simulation: training sample',DP_name] = '-'
    for model,label in zip(models,labels):
        time_rel = model.info['time._simulate_training_sample'] / model.info['time']
        df.loc['\,\,\,simulation: training sample', label] = f"{100*time_rel:.1f}\%"

    df.loc['\,\,\,simulation: out-of-sample $R$',DP_name] = '-'
    for model,label in zip(models,labels):
        time_rel = model.info['time.simulate_R'] / model.info['time']
        df.loc['\,\,\,simulation: out-of-sample $R$', label] = f"{100*time_rel:.1f}\%"

    df.loc['Iterations', DP_name] = '-'
    for model,label in zip(models,labels):
        df.loc['Iterations', label] = model.info['iter']

    df.loc['Avg. policy epochs', DP_name] = '-'
    for model,label in zip(models,labels):
        df.loc['Avg. policy epochs', label] = f"{model.info[('policy_epochs','mean')]:.1f}"

    df.loc['Avg. value epochs', DP_name] = '-'
    for model,label in zip(models,labels):
        df.loc['Avg. value epochs', label] = f"{model.info[('value_epochs','mean')]:.1f}"

    return df

In [ ]:
for D in [1,2,3,8]:

    print(f'{D = }')
    if D == 1:
        DP = models[('DP',f'{D}D','fine')]
        DP_name = 'NEGM+'
    elif D == 2:
        DP = models[('DP',f'{D}D','fine')]
        DP_name = 'NEGM+'
    elif D == 3:
        DP = models[('DP',f'{D}D','rough')]
        DP_name = 'NEGM-'
    else:
        DP = None
        DP_name = 'NEGM'

    models_ = [models[(algoname,f'{D}D')] for algoname in algonames]
    labels = [algolabels[algoname] for algoname in algonames]

    df = create_table(models_,labels,DP=DP,DP_name=DP_name)
    display(df)

    filename = f'{folder}/DurablesModel_table_{D}D.tex'
    df.style.to_latex(filename,hrules=True)  

## 10. <a id='toc10_'></a>[Policy functions](#toc0_)


### 10.1. <a id='toc10_1_'></a>[1D](#toc0_)

#### 10.1.1. <a id='toc10_1_1_'></a>[Chose where to plot policy](#toc0_)


In [39]:
# get means at t
t = 5

means = np.mean(models[('DeepSimulate','1D')].sim.states,axis=1)

# find index in gid closest to mean of p and n
p_index = np.argmin(np.abs(models[('DP','1D')].egm.p_grid - means[t,1]))
n_index = np.argmin(np.abs(models[('DP','1D')].egm.n_grid - means[t,2]))

#### 10.1.2. <a id='toc10_1_2_'></a>[Generate policies in levels](#toc0_)

In [49]:
# GET DL POLICY FUNCTION AT CHOSEN POINTS

algoname = 'DeepSimulate'

# create model to get policy function
# par_dict = {'T':models[(algoname,'1D')].par.T,'D':models[(algoname,'1D')].par.D,'KKT':algoname=='DeepFOC','full':True}
par_dict = {'T':models[(algoname,'1D')].par.T,'D':models[(algoname,'1D')].par.D,'KKT':algoname=='DeepFOC','full':True}
train_dict = {'Nneurons_policy':models[(algoname,'1D')].train.Nneurons_policy}
model = DurablesModelClass(algoname = algoname,par=par_dict,train=train_dict,device='cpu')
model.par.tau = models[(algoname,'1D')].par.tau

# load weights
model.policy_NN.load_state_dict(models[(algoname,'1D')].policy_NN)

# create state-tensor to compute policy on
state_tensor = torch.zeros((models[('DP','1D')].egm.Nm,model.par.Nstates+model.par.T),requires_grad=False)
state_tensor[:,0] = torch.tensor(models[('DP','1D')].egm.m_grid,requires_grad=False)
state_tensor[:,1] = models[('DP','1D')].egm.p_grid[p_index]
state_tensor[:,2] = models[('DP','1D')].egm.n_grid[n_index]
state_tensor[:,2+t] = 1

# compute policy at these points
policy = model.policy_NN(state_tensor)

#  convert policy to levels
c,d,a = compute_d_and_c_from_action(model.par,state_tensor,policy)
c = c.detach().numpy()
d = d.detach().numpy()

In [50]:
def d_consume_all(tau,m,epsilon):
	""" Computation maximum investment level if the consumer spend all cash-on-hand except for epsilon """

	if tau == 0.0:
	
		Delta_max = m - epsilon # if no adjustment cost problem is linear
	
	else:

		a = (-tau)
		b = -1
		c = m - epsilon
		denominator = 2*a
		numerator = (-b - (b**2-4*a*c)**(1/2))
		Delta_max = numerator/denominator

	return Delta_max
def adj_cost(par,Delta):
	""" Adjustment cost """

	return par.nu * (Delta)**2

def compute_d_and_c_from_action_DP(par,state,action,eps=None):
	""" Compute c,d,m_pd from state and action"""

	# a. make array for d
	array_shape =  state.shape[:-1] + (par.D,)
	if type(state) is np.ndarray:
		array_shape =  state.shape[:-1] + (par.D,)
		d_array = np.zeros(array_shape)
	else:
		d_array = torch.zeros(array_shape,dtype=state.dtype,device=state.device)

	# b. loop over durables to get durable choices
	mbar = 0.0 # initialize
	for i_d in range(par.D):

		# i. compute bounds
		d_low = 0.0
		if par.nonnegative_investment:
			d_low = state[...,2+i_d]
		
		if i_d == 0:
			mbar = state[...,0]

		d_high = state[...,2+i_d] + d_consume_all(par.tau,mbar,0.0)

		# ii. add noise to action if needed
		if eps is None:
			action_d = action[...,1+i_d]
		else:
			if type(state) is np.ndarray:
				action_d = np.clip(action[...,1+i_d]+eps[...,1+i_d],0,0.9999)
			else:
				action_d = torch.clamp(action[...,1+i_d]+eps[...,1+i_d],0,0.9999)
		
		# iii. compute durable consumption
		d = d_low+(d_high-d_low)*action_d

		# iv. store durable consumption
		d_array[...,i_d] = d

		# v. compute wealth after durable consumption
		Delta_d = d - state[...,2+i_d]
		mbar = mbar - Delta_d - adj_cost(par,Delta_d)
	
	# c. compute consumption
	if eps is None:
		action_c = action[...,0]
	else:
		if type(state) is np.ndarray:
			action_c = np.clip(action[...,0] + eps[...,0],0,0.9999)
		else:
			action_c = torch.clamp(action[...,0] + eps[...,0],0,0.9999)

	c = mbar*(1-action_c)

	# e. compute a from c
	m_pd = mbar - c

	return c,d_array,m_pd

In [52]:
# GET DP POLICY FUNCTION AT CHOSEN POINTS

# get policy functions
a_policy = models[('DP','1D')].egm.sol_m_pd_fac[t,p_index,n_index]
d_policy = models[('DP','1D')].egm.sol_d1_fac[t,p_index,n_index]

# map policies into same structure as NN_policies
actions = np.zeros((models[('DP','1D')].egm.Nm,2))
actions[:,0] = a_policy
actions[:,1] = d_policy
state_np = state_tensor.numpy()

# get policies in levels
models[('DP','1D')].par.nu = models[('DP','1D')].par.tau 
c_dp,d_dp,m_pd_dp = compute_d_and_c_from_action_DP(models[('DP','1D')].par,state_np,actions)

#### 10.1.3. <a id='toc10_1_3_'></a>[Plot](#toc0_)

In [ ]:
# create figure
fig, ax = plt.subplots(1,1,figsize=(9,6))

# plot policy
ax.plot(models[('DP','1D')].egm.m_grid,c,label='sell',color=colors[0],lw=2)
ax.plot(models[('DP','1D')].egm.m_grid,c_dp,label='sell',color=colors[0],lw=2,ls='--')

ax.set_xlabel('cash-on-hand, $m$')
ax.set_ylabel('consumption, $c$')
ax.set_xlim([0,3])
ax.set_ylim([0.0,1.2])

filepath = f"{folder}/DurablesModel_c_policy_DeepSimulate_D1.svg"
fig.savefig(filepath,bbox_inches='tight')

In [ ]:
# create figure
fig, ax = plt.subplots(1,1,figsize=(9,6))

# plot policy
ax.plot(models[('DP','1D')].egm.m_grid,d,color=colors[0],lw=2, label='DL')
ax.plot(models[('DP','1D')].egm.m_grid,d_dp,label='DP',color=colors[0],lw=2,ls='--')

ax.set_xlabel('cash-on-hand, $m$')
ax.set_ylabel('durable, $d$')
ax.set_xlim([0,3])
ax.set_ylim([0.5,1.2])

# set hline at n_grid[n_index]
ax.axhline(models[('DP','1D')].egm.n_grid[n_index],lw=2,color='black',linestyle='--',label='n')

ax.legend()
filepath = f"{folder}/DurablesModel_d_policy_DeepSimulate_D1.svg"
fig.savefig(filepath,bbox_inches='tight')

### 10.2. <a id='toc10_2_'></a>[8D](#toc0_)

#### 10.2.1. <a id='toc10_2_1_'></a>[Choose where to plot policy](#toc0_)

In [56]:
# get means at t
t = 5

means = np.mean(models[('DeepSimulate','8D')].sim.states,axis=1)
Nm = 100
m_range = np.linspace(1e-8,5,Nm)

#### 10.2.2. <a id='toc10_2_2_'></a>[Get policy in levels](#toc0_)

In [63]:
# GET DL POLICY FUNCTION AT CHOSEN POINTS

algoname = 'DeepSimulate'

# create model to get policy function
par_dict = {'T':models[(algoname,'8D')].par.T,'D':models[(algoname,'8D')].par.D,'KKT':algoname=='DeepFOC','full':True}
train_dict = {'Nneurons_policy':models[(algoname,'8D')].train.Nneurons_policy}
model = DurablesModelClass(algoname = algoname,par=par_dict,train=train_dict,device='cpu')
model.par.nu = model.par.tau = models[(algoname,'8D')].par.tau

# load weights
model.policy_NN.load_state_dict(models[(algoname,'8D')].policy_NN)

# create state-tensor to compute policy on
state_tensor = torch.zeros((Nm,model.par.Nstates+model.par.T),requires_grad=False)
state_tensor[:,0] = torch.tensor(m_range,requires_grad=False)
state_tensor[:,1] = torch.tensor(means[t,1],requires_grad=False)
state_tensor[:,2:2+model.par.D] = torch.tensor(means[t,2:2+model.par.D],requires_grad=False)
state_tensor[:,2+model.par.D+t] = 1

# compute policy at these points
policy = model.policy_NN(state_tensor)

#  convert policy to levels
c,d,a = compute_d_and_c_from_action(model.par,state_tensor,policy)
c = c.detach().numpy()
d = d.detach().numpy()

#### 10.2.3. <a id='toc10_2_3_'></a>[Plot policy](#toc0_)

In [ ]:
# create figure
fig, ax = plt.subplots(1,1,figsize=(9,6))

# plot policy
ax.plot(m_range,c,color=colors[0],lw=2)

ax.set_xlabel('cash-on-hand, $m$')
ax.set_ylabel('consumption, $c$')
ax.set_xlim([0,5])
ax.set_ylim([0.0,1.5])

filepath = f"{folder}/DurablesModel_c_policy_DeepSimulate_D8.svg"
fig.savefig(filepath,bbox_inches='tight')

In [ ]:
# create figure
fig, ax = plt.subplots(1,1,figsize=(9,6))

# plot policy
for i_d in range(model.par.D):
    ax.plot(m_range,d[:,i_d],label=f'$d_{i_d}$',color=colors[i_d],lw=2)

ax.set_xlabel('cash-on-hand,$m$')
ax.set_ylabel('durable goods, $d$')
ax.set_xlim([0,5])
ax.set_ylim([0.0,0.3])
ax.legend(ncol=4)

filepath = f"{folder}/DurablesModel_d_policy_DeepSimulate_D8.svg"
fig.savefig(filepath,bbox_inches='tight')

## 11. <a id='toc11_'></a>[More on convergence](#toc0_)

**Transfer in DeepExplore:**

In [40]:
sim_base = deepcopy(model.sim)

In [ ]:
pos = np.array([])
neg = np.array([1,5,10,20,50,100,500,1000])
transfer_grid = np.concatenate((-np.flip(neg),np.zeros(1),pos))/10_000
R_transfer = np.zeros(transfer_grid.size)

for i,transfer in enumerate(transfer_grid):

    model.sim = deepcopy(sim_base)
    model.sim.states[0,:,0] += transfer
    model.simulate_R()
    R_transfer[i] = model.sim.R

    print(f'transfer = {transfer*10_000:-7.0f}, R = {R_transfer[i]:8.4f}')

**Mimic DP:**

In [42]:
from types import SimpleNamespace
model.sim = sim_base
model.simulate_R()
model.egm = SimpleNamespace()
model.egm.transfer_grid = transfer_grid
model.sim.R_transfer = R_transfer

**Convergence plot:**

In [ ]:
D = 8
Dstr = f'{D}D'
specs = {(algoname,Dstr):algolabels[algoname] for algoname in algonames}
convergence_plot('DurablesModel',models,specs,DP=model,do_transfer=True,
                    xlim=xlim,ylim=[None,1],
                    folder=folder,postfix=f'_{Dstr}_transfer_zoom')